# Assignment 3
Student Name: Ziyi Jiang  |  Student ID: 634926886  |  UPI: zjia631

## Section 1 -- Report

###  Data Representation

#### Training data
The training data (trg.csv) is stored in to two NumPy ndarry:
1. A ndarry stores all the instances' classes (string).
2. A ndarry stores all the instances' abstractes (string).

Reason:
1. Sine we use the training data to do the testing, we need to keep the class column.
2. We can use the index to find the corresponding abstracte of a class, so the id is not needed.

#### Predicting data
The predicting data (tst.csv) is also stored in to two NumPy ndarrys:
1. A ndarry stores all the instances' ids (string).
2. A ndarry stores all the instances' abstractes (string).

Reason: <br/>
Because tst.csv is the predicting data, we need the ids to record the prediction.

### Data Preprocessing

#### Preprocessing method
1. all the data is stored as string in NumPy ndarrys.
2. The id and classe are stored as one string. For example: "1", "A".
3. The abstracte is splited into substrings by " ".

#### Reason
1. The id and class are a single character, so there is no need to split them.
2. We need to count the different words in abstractes to calculate the Naive Bayes result, so the abstracte needs to be splited.

### Extension
The Extension used in this report is Complement Naive Bayes.

#### Reason
Here is the data of the trg.csv:
- Number of each class: {'A': 100, 'B': 1270, 'E': 1729, 'V': 101}
- Total word number of each class: {'A': 21181, 'B': 227337, 'E': 305095, 'V': 18211}

We can see that both the class number and word number of B and E are significantly more than A and V. This will causes the skewed data bias which makes the Standard Naive Bayes classifier predict more to B and E.

However, the Complement Naive Bayes classifier uses the probability of a word occured in other class, which could lower the influence of the skewed data bias and increase the accuracy.

#### Implementation
The Standard Naive Bayes classifier uses the formula: P(class) * P(abstracte|class) <br/>
The Complement Naive Bayes classifier uses the formula: P(class) / P(abstracte|other class) <br/>
Due to the 0 probability problem, I use one and total number of unique words as smoothing parameters for P(abstracte|other class) <br/>
Due to the underflow problem, the code uses np.log() to record the probabilities. <br/>

Detail

Firstly, I recorded 6 parameter:
1. a list of unique class
2. total number of classes
3. total number of each class
4. total number of unique words
5. number of a word in a class
6. total number of words in a class

Then, I calculate P(class|abstracte) of each class:
1. calculate P(class) by using total number of each class / total number of classes
2. calculate P(abstracte|other class) by sum all log(P(word|other class)) of each word in the abstracte:
    - sum the number of the word in all other classes as a
    - sum the total number of words in all other classes as b
    - calculate P(word|other class) by using (a + 1) / (b + total number of unique words)
3. use log(P(class)) - log(P(abstracte|other class)) to get the log-recorded P(class|abstracte).
4. find the maximum log-recorded P(class|abstracte) between all the classes

## Section 2 -- Code

In [1]:
import numpy as np

In [2]:
def train_test_split(x, y, test_size=0.2):
    n_test = int(test_size * len(x))
    x = np.array(x)
    y = np.array(y)
    rng = np.random.default_rng()
    perm = rng.permutation(len(x))
    test = perm[:n_test]
    train = perm[n_test:]
    return x[train], x[test], y[train], y[test]

In [3]:
def cv_split(x, k: int = 5):
    """k-fold cross-validation."""
    rng = np.random.default_rng()
    perm = rng.permutation(len(x))
    for split in np.split(perm, k):
        rest = perm[~np.isin(perm, split, assume_unique=True)]
        yield rest, split

In [32]:
def get_accuracy(result):
    result = np.array(result)
    count = sum(result[:, 0] == result[:, 1])
    accuracy = count / len(result)
    return accuracy

### Standard Naive Bayes Classifier

In [39]:
class NaiveBayesClassifier:

    def __init__(self):
        """
        Unique class is stored as self.unique_class.
        Total number of classes is stored as self.total_classes_number.
        Total number of each class is stored in self.class_number_dict.
        Total number of unique words is stored as self.unique_word_number.
        Total number of a word in a class is stored in self.class_word_number_dict.
        otal number of a word in a class is stored in self.class_total_words_dict.
        """
        self.unique_classes = []
        self.total_classes_number = 0
        self.class_number_dict = {}
        self.unique_word_number = 0
        self.class_word_number_dict = {}
        self.class_total_words_dict = {}

    def fit(self, abstracts, classes):
        classes = np.array(classes)
        abstracts = np.array(abstracts)
        # unique class
        self.unique_classes = sorted(set(classes))

        # total number of classes
        self.total_classes_number = len(classes)

        # total number of each class
        c, counts = np.unique(classes, return_counts=True)
        self.class_number_dict = dict(zip(c, counts))

        # total number of unique words
        all_text = ""
        for text in abstracts:
            all_text = all_text + text + " "
        all_text = all_text.split()
        unique_word = set(all_text)
        self.unique_word_number = len(unique_word)

        # total number of a word in a class
        self.class_word_number_dict = {}
        class_text_dict = {}
        for c in self.unique_classes:
            class_text_dict[c] = ""
        for i in range(len(abstracts)):
            c = classes[i]
            text = abstracts[i]
            class_text_dict[c] = class_text_dict[c] + text + " "
        for c in class_text_dict:
            text = np.array(class_text_dict[c].split())
            word, counts = np.unique(text, return_counts=True)
            self.class_word_number_dict[c] = dict(zip(word, counts))

        # total number of words in a class
        self.class_total_words_dict = {}
        for c in class_text_dict:
            text = class_text_dict[c].split()
            self.class_total_words_dict[c] = len(text)

    def predict(self, abstracts, ids):
        result = []
        for i in range(len(ids)):
            pred_id = ids[i]
            pred_abs = abstracts[i].split()
            id_result_prob = []
            for c in self.unique_classes:
                prob_c = np.log(self.class_number_dict[c] / self.total_classes_number)
                prob_x = 0
                word_number_dict = self.class_word_number_dict[c]
                total_words_number = self.class_total_words_dict[c]
                for word in pred_abs:
                    if word in word_number_dict:
                        word_number = word_number_dict[word]
                    else:
                        word_number = 0
                    prob_word = np.log((word_number + 1) / (total_words_number + self.unique_word_number))
                    prob_x = prob_x + prob_word
                prob = prob_c + prob_x
                id_result_prob.append([c, prob])

            id_result_prob = sorted(id_result_prob, key=lambda x: x[1])
            result.append((pred_id, id_result_prob[-1][0]))

        return result

### Navie Bayes Classifier with Extension

#### Complement Naive Bayes

In [6]:
class ComplementNaiveBayesClassifier:

    def __init__(self):
        self.unique_classes = []
        self.total_classes_number = 0
        self.class_number_dict = {}
        self.unique_word_number = 0
        self.class_word_number_dict = {}
        self.class_total_words_dict = {}

    def fit(self, abstracts, classes):
        classes = np.array(classes)
        abstracts = np.array(abstracts)
        
        self.unique_classes = sorted(set(classes))

        self.total_classes_number = len(classes)

        c, counts = np.unique(classes, return_counts=True)
        self.class_number_dict = dict(zip(c, counts))

        all_text = ""
        for text in abstracts:
            all_text = all_text + text + " "
        all_text = all_text.split()
        unique_word = set(all_text)
        self.unique_word_number = len(unique_word)

        self.class_word_number_dict = {}
        class_text_dict = {}
        for c in self.unique_classes:
            class_text_dict[c] = ""
        for i in range(len(abstracts)):
            c = classes[i]
            text = abstracts[i]
            class_text_dict[c] = class_text_dict[c] + text + " "
        for c in class_text_dict:
            text = np.array(class_text_dict[c].split())
            word, counts = np.unique(text, return_counts=True)
            self.class_word_number_dict[c] = dict(zip(word, counts))

        self.class_total_words_dict = {}
        for c in class_text_dict:
            text = class_text_dict[c].split()
            self.class_total_words_dict[c] = len(text)

    def predict(self, abstracts, ids):
        result = []
        for i in range(len(ids)):
            pred_id = ids[i]
            pred_abs = abstracts[i].split()
            id_result_prob = []
            for c in self.unique_classes:
                prob_c = np.log(self.class_number_dict[c] / self.total_classes_number)
                prob_x = 0
                """
                Extension
                """
                not_cs = []
                word_number_in_ncs = []
                total_words_number = 0
                for nc in self.unique_classes:
                    if nc != c:
                        not_cs.append(nc)
                        word_number_in_ncs.append(self.class_word_number_dict[nc])
                        total_words_number += self.class_total_words_dict[nc]  
                for word in pred_abs:
                    word_number = 0
                    for word_number_dict in word_number_in_ncs:
                        if word in word_number_dict:
                            word_number += word_number_dict[word]
                    prob_word = np.log((word_number + 1) / (total_words_number + self.unique_word_number))
                    prob_x = prob_x + prob_word
                prob = prob_c - prob_x
                id_result_prob.append([c, prob])
            id_result_prob = sorted(id_result_prob, key=lambda x: x[1])
            result.append((pred_id, id_result_prob[-1][0]))

        return result

### Test

#### Standard Naive Bayes

In [7]:
classes = []
train_abstracts = []
training_data = np.genfromtxt("trg.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(1, 2))
classes = training_data[:, 0]
train_abstracts = training_data[:, 1]

train_x, test_x, train_y, test_y = train_test_split(train_abstracts, classes)

In [33]:
nbc =NaiveBayesClassifier()
nbc.fit(train_x, train_y)
result = nbc.predict(test_x, test_y)

nbc_accuracy = get_accuracy(result)
print(nbc_accuracy)

0.94375


In [34]:
cnbc =ComplementNaiveBayesClassifier()
cnbc.fit(train_x, train_y)
result = cnbc.predict(test_x, test_y)

cnbc_accuracy = get_accuracy(result)
print(cnbc_accuracy)

0.95875


In [36]:
nbc_scores = []
cnbc_scores = []
for i in range(10):
    train_x, test_x, train_y, test_y = train_test_split(train_abstracts, classes)
    
    nbc =NaiveBayesClassifier()
    nbc.fit(train_x, train_y)
    nbc_result = nbc.predict(test_x, test_y)
    nbc_accuracy = get_accuracy(nbc_result)
    nbc_scores.append(nbc_accuracy)
    
    cnbc =ComplementNaiveBayesClassifier()
    cnbc.fit(train_x, train_y)
    cnbc_result = cnbc.predict(test_x, test_y)
    cnbc_accuracy = get_accuracy(cnbc_result)
    cnbc_scores.append(cnbc_accuracy)

print("Standard mean:", np.mean(nbc_scores))
print("Standard std:", np.std(nbc_scores))
print("Complement mean:", np.mean(cnbc_scores))
print("Complement std:", np.std(cnbc_scores))

Standard mean: 0.9403749999999998
Standard std: 0.006047985201701463
Complement mean: 0.961375
Complement std: 0.004919667163538611


### Final Result of tst.csv

In [11]:
train_classes = []
train_abstracts = []
training_data = np.genfromtxt("trg.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(1, 2))
train_classes = training_data[:, 0]
train_abstracts = training_data[:, 1]

ids = []
abstracts = []
data = np.genfromtxt("tst.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(0, 1))
ids = data[:, 0]
abstracts = data[:, 1]

nbc = NaiveBayesClassifier()
nbc.fit(train_abstracts, train_classes)
result = nbc.predict(abstracts, ids)

f = open("nbc_result.csv", "a")
f.write("id,class\n")
for r in result:
    line = r[0] + "," + r[1] + "\n"
    f.write(line)
f.close()

In [12]:
train_classes = []
train_abstracts = []
training_data = np.genfromtxt("trg.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(1, 2))
train_classes = training_data[:, 0]
train_abstracts = training_data[:, 1]

ids = []
abstracts = []
data = np.genfromtxt("tst.csv", delimiter=",",  skip_header=1, dtype=str, usecols=(0, 1))
ids = data[:, 0]
abstracts = data[:, 1]

cnbc = ComplementNaiveBayesClassifier()
cnbc.fit(train_abstracts, train_classes)
result = cnbc.predict(abstracts, ids)
f = open("cnbc_result.csv", "a")
f.write("id,class\n")
for r in result:
    line = r[0] + "," + r[1] + "\n"
    f.write(line)
f.close()

In [38]:
print("Class number:", cnbc.class_number_dict)
print("Class word number:", cnbc.class_total_words_dict)

Class number: {'A': 100, 'B': 1270, 'E': 1729, 'V': 101}
Class word number: {'A': 21181, 'B': 227337, 'E': 305095, 'V': 18211}
